In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.hdfcbank.com/personal/pay/cards/credit-cards'
url_short = 'https://www.hdfcbank.com'

In [3]:
def fetch_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [4]:
html_content = fetch_page(url)
soup = BeautifulSoup(html_content, 'html.parser')
main = soup.find("div", class_="bp-area c-template-hdfc-middle-section--area")
categories = main.find_all("div", class_="bp-container re-order-container c-template-hdfc-portal-accordion mainDivAcc bp-widget bp-ui-dragRoot")
icons = ["accepted","aeroplane","membership","shopping","reward_points","food","donations","travel_insurance_hover","invest","cash","gift_rewards","content","Deposits","fuel","emi","benefit","interest","save","convenience","wallet","transfer","feature"]

In [5]:
def add_features(feature):
  for icon in icons:
    if icon in feature.find("img")["value"]:
      feature_con = feature.text
      return icon, feature_con
  return "other", feature.text

In [6]:
cards = []
for category in categories:
  category_name = category.find("div", class_="divContent").text.strip()
  card_info = category.find("div", class_="bp-area c-template-hdfc-portal-accordion--area col-xs-12 col-sm-12 col-md-12 col-lg-12 contentacc")
  cards_obj = card_info.find_all("div", class_="bp-widget bp-ui-dragRoot")
  for card in cards_obj:
    features = {}
    name = card.find("span", class_="card-name").text.strip()
    features["name"] = name
    features["category"] = category_name
    for li in card.find_all('li'):
      icon, feature_con = add_features(li)
      if icon in features:
        features[icon] += "; " + feature_con
      else:
        features[icon] = feature_con
    link_obj = card.find_all("div", class_="btnParent")
    for link in link_obj:
      if link.find("a", title="Apply Online"):
        apply_link = link.find("a", title="Apply Online")["href"]
      else:
        apply_link = None
      if link.find("a", title="Know More"):
        info_link = url_short + link.find("a", title="Know More")["href"]
      else:
        info_link = None
    features["apply_link"] = apply_link
    features["info_link"] = info_link
    cards.append(features)

In [7]:
cards_df = pd.DataFrame(cards)

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

# cards_df.to_json('/content/drive/MyDrive/data/yxt_credit_card_HDFC.json', orient='index')